In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
import os

In [3]:
%pwd

'c:\\Users\\deyar\\OneDrive\\Desktop\\cnnClassifier\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\deyar\\OneDrive\\Desktop\\cnnClassifier'

In [6]:
from src.cnnClassifier.logger import logging
from src.cnnClassifier.exception import customexception

In [7]:
from src.cnnClassifier.constants.cont import CONFIG_FILE_PATH,PARAMS_FILE_PATH

In [8]:
import yaml
from box import ConfigBox

In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTraining:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    EPOCHS: int
    LEARNING_RATE: float
    class_mode: str
    color_mode: str
    BATCH_SIZE: int

In [10]:
import sys

In [11]:
class ConfigManager:
    def __init__(self,paramspath=PARAMS_FILE_PATH,configpath=CONFIG_FILE_PATH):
        try:
         self.config=ConfigBox(yaml.safe_load(open(configpath)))
         self.params=ConfigBox(yaml.safe_load(open(paramspath)))
         os.makedirs(self.config.training.root_dir,exist_ok=True)
         logging.info(f"{self.config.training.root_dir} has been created")
        except Exception as e:
         logging.info("dir is not created")
         raise customexception(e,sys)
    def get_model_Training_config(self):
        param_config=self.params
        config=self.config.prepare_base_model
        train=self.config.training
        model_training_config=ModelTraining(root_dir=train.root_dir,
            trained_model_path=train.trained_model_path,
            updated_base_model_path=config.updated_base_model_path,
            EPOCHS=param_config.EPOCHS,
            LEARNING_RATE=param_config.LEARNING_RATE,
            BATCH_SIZE=param_config.BATCH_SIZE,
            class_mode=param_config.class_mode,
            color_mode=param_config.color_mode)
        return model_training_config

In [12]:
n=ConfigManager()

In [13]:
nn=n.get_model_Training_config()

In [14]:
nn

ModelTraining(root_dir='artifacts/training', trained_model_path='artifacts/training/model.h5', updated_base_model_path='artifacts/prepare_base_model/base_model_updated.h5', EPOCHS=10000, LEARNING_RATE=0.001, class_mode='categorical', color_mode='rgb', BATCH_SIZE=16)

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.metrics import Precision, Recall
from keras.callbacks import EarlyStopping

In [ ]:
class getfinal__model():
  def start_traing(self,p:ModelTraining):
    self.model_path=(p.trained_model_path)
    self.EPOCHS=p.EPOCHS
    self.LEARNING_RATE=p.LEARNING_RATE
    self.class_mode=p.class_mode
    self.color_mode=p.color_mode
    self.BATCH_SIZE=p.BATCH_SIZE
    self.updated_base_model_path=p.updated_base_model_path
    train_datagen=ImageDataGenerator(rotation_range=30,  #create data augumentation
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[.1,1],
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    vertical_flip=True,
    rescale=1/255)
    test_datagen=ImageDataGenerator(rescale=1/255) #normalize
    valid_datagen=ImageDataGenerator(rescale=1/255) #normalize
    train_data_path = Path("artifacts") / "data_ingestion" / "Data" / "train"
    test_data_path = Path("artifacts") / "data_ingestion" / "Data" / "test"
    valid_data_path = Path("artifacts") / "data_ingestion" / "Data" / "valid"
    train_data = train_datagen.flow_from_directory(train_data_path,
    target_size=(224,224),
    color_mode=self.color_mode,
    class_mode=self.class_mode,  #train data generator
    batch_size=self.BATCH_SIZE,
    shuffle=True)
    self.test_data = test_datagen.flow_from_directory(test_data_path,
    target_size=(224,224),
    color_mode=self.color_mode,
    class_mode=self.class_mode,  #train data generator
    batch_size=self.BATCH_SIZE,
    shuffle=True)
    val_data = valid_datagen.flow_from_directory(valid_data_path,
    target_size=(224,224),
    color_mode=self.color_mode,
    class_mode=self.class_mode,  #train data generator
    batch_size=self.BATCH_SIZE,
    shuffle=True)
    logging.info("data has been ready for traing")
    logging.info(f'{train_data.class_indices}')
    logging.info(f'test data{self.test_data.class_indices}')
    model=load_model(self.updated_base_model_path)
    logging.info("model has been loaded")
    optimizer = RMSprop(self.LEARNING_RATE)
    model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy',Precision(), Recall()])
    z=EarlyStopping(monitor="val_precision_5",
    min_delta=0.0001,
    patience=3,
    verbose=0,
    mode="max",
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0)
    history=model.fit(train_data,epochs=self.EPOCHS,validation_data=val_data,callbacks=z)
    return model.save(self.model_path)
  def model_eval(self,p:ModelTraining):
    model=load_model(self.model_path)
    self.eval=model.evaluate(self.test_data)
    print(self.eval)

In [ ]:
train=getfinal__model()

In [83]:
train.start_traing(nn)

Found 613 images belonging to 4 classes.


Found 315 images belonging to 4 classes.
Found 72 images belonging to 4 classes.
Epoch 1/10000
39/39 ━━━━━━━━━━━━━━━━━━━━ 35s 770ms/step - accuracy: 0.3215 - loss: 1.5222 - precision_5: 0.3521 - recall_5: 0.1368 - val_accuracy: 0.3889 - val_loss: 1.3091 - val_precision_5: 0.0000e+00 - val_recall_5: 0.0000e+00
Epoch 2/10000
39/39 ━━━━━━━━━━━━━━━━━━━━ 28s 704ms/step - accuracy: 0.4196 - loss: 1.2913 - precision_5: 0.5805 - recall_5: 0.1751 - val_accuracy: 0.3611 - val_loss: 1.6318 - val_precision_5: 0.9167 - val_recall_5: 0.1528
Epoch 3/10000
39/39 ━━━━━━━━━━━━━━━━━━━━ 27s 697ms/step - accuracy: 0.4033 - loss: 1.3063 - precision_5: 0.4914 - recall_5: 0.1301 - val_accuracy: 0.3889 - val_loss: 1.3235 - val_precision_5: 1.0000 - val_recall_5: 0.1389
Epoch 4/10000
39/39 ━━━━━━━━━━━━━━━━━━━━ 27s 695ms/step - accuracy: 0.4493 - loss: 1.2544 - precision_5: 0.6459 - recall_5: 0.1659 - val_accuracy: 0.2917 - val_loss: 1.3290 - val_precision_5: 0.7647 - val_recall_5: 0.1806
Epoch 5/10000
39/39 ━━━

In [84]:
train.model_eval(nn)

20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 610ms/step - accuracy: 0.4568 - loss: 1.2446 - precision_5: 1.0000 - recall_5: 0.1977
[1.2807774543762207, 0.4253968298435211, 1.0, 0.17142857611179352]
